In [1]:
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.60:4040
SparkContext available as 'sc' (version = 3.0.0-preview2, master = local[*], app id = local-1593940868680)
SparkSession available as 'spark'


import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils


In [2]:
val data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt")
data

data: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[6] at map at MLUtils.scala:86
res0: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[6] at map at MLUtils.scala:86


In [3]:
data.collect()(0)

res1: org.apache.spark.mllib.regression.LabeledPoint = (0.0,(692,[127,128,129,130,131,154,155,156,157,158,159,181,182,183,184,185,186,187,188,189,207,208,209,210,211,212,213,214,215,216,217,235,236,237,238,239,240,241,242,243,244,245,262,263,264,265,266,267,268,269,270,271,272,273,289,290,291,292,293,294,295,296,297,300,301,302,316,317,318,319,320,321,328,329,330,343,344,345,346,347,348,349,356,357,358,371,372,373,374,384,385,386,399,400,401,412,413,414,426,427,428,429,440,441,442,454,455,456,457,466,467,468,469,470,482,483,484,493,494,495,496,497,510,511,512,520,521,522,523,538,539,540,547,548,549,550,566,567,568,569,570,571,572,573,574,575,576,577,578,594,595,596,597,598,599,600,601,602,603,604,622,623,624,625,626,627,628,629,630,651,652,653,654,655,656,657],[51.0,159.0,253.0,159.0,50...


In [4]:
data.collect()(1)

res2: org.apache.spark.mllib.regression.LabeledPoint = (1.0,(692,[158,159,160,161,185,186,187,188,189,213,214,215,216,217,240,241,242,243,244,245,267,268,269,270,271,295,296,297,298,322,323,324,325,326,349,350,351,352,353,377,378,379,380,381,404,405,406,407,408,431,432,433,434,435,459,460,461,462,463,486,487,488,489,490,514,515,516,517,518,542,543,544,545,569,570,571,572,573,596,597,598,599,600,601,624,625,626,627,652,653,654,655,680,681,682,683],[124.0,253.0,255.0,63.0,96.0,244.0,251.0,253.0,62.0,127.0,251.0,251.0,253.0,62.0,68.0,236.0,251.0,211.0,31.0,8.0,60.0,228.0,251.0,251.0,94.0,155.0,253.0,253.0,189.0,20.0,253.0,251.0,235.0,66.0,32.0,205.0,253.0,251.0,126.0,104.0,251.0,253.0,184.0,15.0,80.0,240.0,251.0,193.0,23.0,32.0,253.0,253.0,253.0,159.0,151.0,251.0,251.0,251.0,39.0,48.0,221....


In [5]:
val splits = data.randomSplit(Array(0.6, 0.4), seed = 111)

val (trainingData, testData) = (splits(0), splits(1))

splits: Array[org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint]] = Array(MapPartitionsRDD[7] at randomSplit at <console>:34, MapPartitionsRDD[8] at randomSplit at <console>:34)
trainingData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[7] at randomSplit at <console>:34
testData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[8] at randomSplit at <console>:34


In [6]:
trainingData.count()

res3: Long = 63


In [7]:
testData.count()

res4: Long = 37


In [8]:
data.count()

res5: Long = 100


## ENTRENAMIENTO DE MULTIPLES MODELOS

#### /////////////////////////////////////////////  Regesión Logística

In [9]:
val model_logistic = new LogisticRegressionWithLBFGS()
  .setNumClasses(2)
  .run(trainingData)

model_logistic: org.apache.spark.mllib.classification.LogisticRegressionModel = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 0.0, numFeatures = 692, numClasses = 2, threshold = 0.5


In [10]:
val predictionAndLabels_logistic = testData.map { case LabeledPoint(label, features) =>
  val prediction = model_logistic.predict(features)
  (prediction, label)
}

predictionAndLabels_logistic: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[65] at map at <console>:36


In [11]:
val metrics_logistic = new BinaryClassificationMetrics(predictionAndLabels_logistic)
val auROC_logistic = metrics_logistic.areaUnderROC()

metrics_logistic: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@6668e4b7
auROC_logistic: Double = 0.9782608695652174


#### ///////////////////////////////////////////// SVM

In [12]:
val numIterations = 100
val model_svm = SVMWithSGD.train(
    trainingData, numIterations
)

numIterations: Int = 100
model_svm: org.apache.spark.mllib.classification.SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 692, numClasses = 2, threshold = 0.0


In [13]:
val scoreAndLabels_svm = testData.map { point =>
  val score = model_svm.predict(point.features)
  (score, point.label)
}

scoreAndLabels_svm: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[283] at map at <console>:36


In [14]:
val metrics_svm = new BinaryClassificationMetrics(scoreAndLabels_svm)
val auROC_svm = metrics_svm.areaUnderROC()

metrics_svm: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@461a5372
auROC_svm: Double = 1.0


#### ///////////////////////////////////////////// RANDOM FOREST

In [15]:
val numClasses = 2
val categoricalFeaturesInfo = Map[Int, Int]()
val numTrees = 30 // Use more in practice.
val featureSubsetStrategy = "auto" // Let the algorithm choose.
val impurity = "gini"
val maxDepth = 4
val maxBins = 32

val model_forest = RandomForest.trainClassifier(trainingData, numClasses, categoricalFeaturesInfo,
  numTrees, featureSubsetStrategy, impurity, maxDepth, maxBins)

numClasses: Int = 2
categoricalFeaturesInfo: scala.collection.immutable.Map[Int,Int] = Map()
numTrees: Int = 30
featureSubsetStrategy: String = auto
impurity: String = gini
maxDepth: Int = 4
maxBins: Int = 32
model_forest: org.apache.spark.mllib.tree.model.RandomForestModel =
TreeEnsembleModel classifier with 30 trees


In [16]:
val scoreAndLabels_forest = testData.map { point =>
  val prediction_forest = model_forest.predict(point.features)
  (prediction_forest, point.label)
}

scoreAndLabels_forest: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[316] at map at <console>:36


In [17]:
val metrics_forest = new BinaryClassificationMetrics(scoreAndLabels_forest)
val auROC_forest = metrics_forest.areaUnderROC()

metrics_forest: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@6272f352
auROC_forest: Double = 1.0


### ///////////////////////////////////////////// Persistencia de modelos

In [19]:
val path_save = "target/model_forest2"

model_forest.save(sc, path_save)

val model_forest_load = RandomForestModel.load(
  sc, path_save
)

path_save: String = target/model_forest2
model_forest_load: org.apache.spark.mllib.tree.model.RandomForestModel =
TreeEnsembleModel classifier with 30 trees


In [20]:
val scoreAndLabels_forest_load = testData.map { point =>
  val prediction_forest_load = model_forest_load.predict(point.features)
  (prediction_forest_load, point.label)
}

scoreAndLabels_forest_load: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[355] at map at <console>:36


In [21]:
val metrics_forest_load = new BinaryClassificationMetrics(scoreAndLabels_forest_load)
val auROC_forest_load = metrics_forest_load.areaUnderROC()

metrics_forest_load: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@7b2e75f0
auROC_forest_load: Double = 1.0
